In [1]:
import pandas as pd
import os
import pickle

In [2]:
df = pd.read_excel("../../Dataset/All_Transcriptions_WithSpeakers.xlsx")

In [3]:
df

,file_name,segment_id,segment_text,segment_start,segment_end,word,word_start,word_end,speaker
0,MagnusCarlson_542_599.mp4,0,I'm going to name a sport.,0.00,0.94,I'm,0.00,0.18,Levy Rozman
1,MagnusCarlson_542_599.mp4,0,I'm going to name a sport.,0.00,0.94,going,0.18,0.28,Levy Rozman
2,MagnusCarlson_542_599.mp4,0,I'm going to name a sport.,0.00,0.94,to,0.28,0.36,Levy Rozman
3,MagnusCarlson_542_599.mp4,0,I'm going to name a sport.,0.00,0.94,name,0.36,0.46,Levy Rozman
4,MagnusCarlson_542_599.mp4,0,I'm going to name a sport.,0.00,0.94,a,0.46,0.66,Levy Rozman
...,...,...,...,...,...,...,...,...,...
14397,PiersMorgan_1_165_368.mp4,97,Your son has never called you daddy for a reason.,199.48,201.74,a,201.36,201.46,Piers
14398,PiersMorgan_1_165_368.mp4,97,Your son has never called you daddy for a reason.,199.48,201.74,reason.,201.46,201.74,Piers
14399,PiersMorgan_1_165_368.mp4,98,You know why?,201.74,202.42,You,201.74,202.18,Piers
14400,PiersMorgan_1_165_368.mp4,98,You know why?,201.74,202.42,know,202.18,202.24,Piers


In [5]:
file_name_list = df["file_name"].unique()

for file_name in file_name_list:
    temp = df[df["file_name"] == file_name].reset_index(drop=True)
    temp.to_csv(f"/vol3/sunil/AVA-AVD/dataset/csv/{os.path.splitext(file_name)[0]}.csv", index=False, sep="\t")

In [4]:
class VideoFile:
    def __init__(self, name, save_name, start=-1, end=-1, duration=-1, ground_truth_type="der", ground_truth_file=None):
        self.name = name
        self.save_name = save_name
        self.start = start
        self.end = end
        self.duration = duration
        self.ground_truth_type = ground_truth_type
        self.ground_truth_file = ground_truth_file

In [5]:
video_file_1 = VideoFile(
    name="MagnusCarlson_542_599",
    save_name="MagnusCarlson_542_599",
    ground_truth_type="wder",
    ground_truth_file="/vol3/sunil/my_dataset/dataset/csv/MagnusCarlson_542_599.csv"
)
video_file_2 = VideoFile(
    name="StarTalk_Questions_831_5_924",
    save_name="StarTalk_Questions_831_5_924",
    ground_truth_type="wder",
    ground_truth_file="/vol3/sunil/my_dataset/dataset/csv/StarTalk_Questions_831_5_924.csv"
)

In [43]:
ado = pickle.load(open("../../output/run_output_all_videos/audio_diarization_output.pckl", "rb"))
vdo = pickle.load(open("../../output/run_output_all_videos/video_diarization_output.pckl", "rb"))

In [29]:
def find_overlap(intervals1, intervals2):
    overlap = 0
    total_duration1 = 0
    total_duration2 = 0

    # Calculate the total duration of intervals in intervals1
    for start, end in intervals1:
        total_duration1 += end - start

    # Calculate the total duration of intervals in intervals2 and find the overlap
    for start, end in intervals2:
        total_duration2 += end - start
        for s1, e1 in intervals1:
            common_start = max(s1, start)
            common_end = min(e1, end)
            if common_start < common_end:
                overlap += common_end - common_start

    # Calculate the percentage of overlap with respect to intervals1
    if total_duration1 == 0:
        percentage_overlap1 = 0
    else:
        percentage_overlap1 = (overlap / total_duration1) * 100

    # Calculate the percentage of overlap with respect to intervals2
    if total_duration2 == 0:
        percentage_overlap2 = 0
    else:
        percentage_overlap2 = (overlap / total_duration2) * 100

    return percentage_overlap1, percentage_overlap2

def get_word_to_speaker_mapping(word_start, word_end, final_output):
    final_speaker = None
    max_overlap = 0

    for speaker_key in final_output.keys():
        word_overlap, _ = find_overlap([(word_start, word_end)], final_output[speaker_key])
        if word_overlap > max_overlap:
            final_speaker = speaker_key
            max_overlap = word_overlap

    if final_speaker is None:
        final_speaker = "Unassigned"

    return final_speaker

In [24]:
def match_output(output_df):

    assigned_speaker_list = output_df["assigned_speaker"].unique()
    speaker_match = {}
    speaker_count = {}

    for assigned_speaker in assigned_speaker_list:
        if assigned_speaker == "Unassigned":
            speaker_match[assigned_speaker] = "Unassigned"
        else:
            speaker_match[assigned_speaker] = (
                output_df[output_df["assigned_speaker"] == assigned_speaker]["speaker"]
                .value_counts()
                .idxmax()
            )
            speaker_count[assigned_speaker] = len(
                output_df[output_df["assigned_speaker"] == assigned_speaker]
            )

    temp_count = {}

    for speaker_pred, speaker_actual in speaker_match.items():
        if speaker_actual in temp_count.keys():
            temp_count[speaker_actual].append(speaker_pred)
        else:
            temp_count[speaker_actual] = [speaker_pred]

    for speaker_actual, assigned_speakers in temp_count.items():
        if len(assigned_speakers) > 1:
            max_count = 0
            max_speaker = None
            for speaker in assigned_speakers:
                if speaker_count[speaker] > max_count:
                    if max_speaker is None:
                        max_speaker = speaker
                        max_count = speaker_count[speaker]
                    else:
                        speaker_match[max_speaker] = "Unassigned"
                        max_speaker = speaker
                        max_count = speaker_count[speaker]
                else:
                    speaker_match[speaker] = "Unassigned"
            speaker_match[max_speaker] = speaker_actual

    output_df["matched_speaker"] = output_df.apply(
        lambda row: speaker_match[row["assigned_speaker"]],
        axis=1,
    )

    return output_df

In [46]:
def compute_performance(output_df):

    def speaker_match(speaker, assigned_speaker, matched_speaker):
        if assigned_speaker == "Unassigned":
            return "Missed"
        elif matched_speaker == "Unassigned" or matched_speaker != speaker:
            return "Confusion"
        else:
            return "Correct"

    output_df["matched_result"] = output_df.apply(
        lambda row: speaker_match(row["speaker"], row["assigned_speaker"], row["matched_speaker"]),
        axis=1,
    )

    return output_df["matched_result"].value_counts().to_dict()

In [47]:
file_name = video_file_2.save_name
gt_path = video_file_2.ground_truth_file

df = pd.read_csv(gt_path, sep="\t")
df["assigned_speaker"] = df.apply(
    lambda row: (
        get_word_to_speaker_mapping(
            row["word_start"],
            row["word_end"],
            vdo[file_name],
        )
        if row["word_end"] > row["word_start"]
        else get_word_to_speaker_mapping(
            row["word_start"],
            row["word_end"] + 0.01,
            vdo[file_name],
        )
    ),
    axis=1,
)

matched_df = match_output(df)
compute_performance(matched_df)

{'Correct': 190, 'Missed': 60, 'Confusion': 6}

In [13]:
df = pd.read_excel("/vol3/sunil/MSDWild/SelectedRTTMs.xlsx", header=None)
df.columns = ["temp", "file_name", "channel", "start", "duration", "NA_1", "NA_2", "speaker_label", "NA_3", "NA_4"]
df["file_name"] = df["file_name"].apply(lambda x: "%05d" % x)
df = df.fillna("<NA>")
df

,temp,file_name,channel,start,duration,NA_1,NA_2,speaker_label,NA_3,NA_4
0,SPEAKER,00077,0,0.41200,5.76882,<NA>,<NA>,3,<NA>,<NA>
1,SPEAKER,00077,0,4.60619,0.73181,<NA>,<NA>,0,<NA>,<NA>
2,SPEAKER,00077,0,5.82738,0.61463,<NA>,<NA>,0,<NA>,<NA>
3,SPEAKER,00077,0,6.55716,8.14864,<NA>,<NA>,3,<NA>,<NA>
4,SPEAKER,00077,0,9.09700,0.66405,<NA>,<NA>,4,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...
2025,SPEAKER,03135,0,40.88300,3.56569,<NA>,<NA>,1,<NA>,<NA>
2026,SPEAKER,03135,0,44.36797,2.37703,<NA>,<NA>,0,<NA>,<NA>
2027,SPEAKER,03135,0,46.67878,2.47322,<NA>,<NA>,1,<NA>,<NA>
2028,SPEAKER,03135,0,47.89300,0.38900,<NA>,<NA>,0,<NA>,<NA>


In [12]:
df["file_name"].unique()

array(['00077', '00170', '00340', '00417', '00430', '00616', '00663',
       '00760', '00877', '00978', '01036', '01166', '01337', '01491',
       '01596', '01649', '01744', '01954', '01984', '01997', '02071',
       '02162', '02239', '02258', '02311', '02409', '02707', '02846',
       '03010', '03020', '00017', '00099', '00200', '00369', '00481',
       '00583', '00689', '00740', '00817', '00852', '00942', '00979',
       '01009', '01158', '01226', '01273', '01284', '01290', '01329',
       '01378', '01385', '01418', '01725', '01763', '01764', '01827',
       '01875', '02053', '02057', '02093', '02150', '02161', '02187',
       '02295', '02405', '02457', '02513', '02515', '02564', '02606',
       '02619', '02693', '02743', '02785', '02848', '02901', '02942',
       '02972', '02978', '03135'], dtype=object)

In [15]:
for file_name in df["file_name"].unique():
    temp = df[df["file_name"] == file_name]
    temp.to_csv(f"/vol3/sunil/MSDWild/rttms/{file_name}.rttm", index=False, header=False, sep=" ")